# 1. 导入所需要的模块并设置语料的路径

In [1]:
import torch
import torch.nn as nn
import jieba
import matplotlib.pyplot as plt

en_name = './corpus/train/EN.txt'
cn_name = './corpus/train/CN.txt'

# 2. 从文件中读取数据并且使用jieba分词，随后生成词表

In [2]:
def file2tokens(file_name):
    r"""
    :param file_name: 
    :return : tokens in lists
    """
    with open(file_name, 'r') as file:
        data = file.read()
        data = data.split('\n')
        data = [list(jieba.cut(t)) for t in data]
        data = [t for t in data if len(t) > 5]
        return data, max([len(i) for i in data])


def tokens2vocab(data, target=None):
    r"""
    根据分词后的文本生成对应的词表(vocab)和相关数据
    :param data:generated by file2tokens
    :param target: should be 'tgt' or 'src'
    :return: 
    """
    if target == 'src':
        vocab = {'P': 0}
        for sentence in data:
            for token in sentence:
                if not token in vocab:
                    vocab[token] = len(vocab)
        idx2token = {i: t for i, t in enumerate(vocab)}
        vocab_size = len(vocab)
    elif target == 'tgt':
        vocab = {'P': 0, 'S': 1, 'E': 2}
        for sentence in data:
            for token in sentence:
                if not token in vocab:
                    vocab[token] = len(vocab)
        idx2token = {i: t for i, t in enumerate(vocab)}
        vocab_size = len(vocab)
    else:
        raise ValueError("invalid param about target!")
    return vocab, idx2token, vocab_size

# 3. 生成transformer使用的Tensor序列

In [4]:
def make_data(token_lists: (list, list), size: (int, int)):
    r"""
    把分词后的文本转化成下标序列。本函数同时实现了padding的功能。
    
    :param token_lists:输入由file2tokens生成的src_tokens和tgt_tokens
    :param size: 
    :return: [LongTensor, LongTensor, LongTensor]
    """
    pass


src_tokens, src_max_len = file2tokens(en_name)
src_vocab, idx2src, src_vocab_size = tokens2vocab(src_tokens, 'src')
print(src_max_len)

97
